In [1]:
!git clone https://github.com/vladislavneon/RuBQ.git

Cloning into 'RuBQ'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 106 (delta 56), reused 29 (delta 15), pack-reused 0
Receiving objects: 100% (106/106), 14.13 MiB | 11.14 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
import json

filepath = "/content/RuBQ/RuBQ_2.0/RuBQ_2.0_test.json"
with open(filepath, mode="r", encoding="utf-8") as input:
  lines = input.readlines()
res_json = json.loads(''.join(lines))
dataset_temp = [x["question_text"] for x in res_json]


In [3]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
!unzip quora_question_pairs_rus.csv.zip

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=fa86cf63f89eaf91d40ae079806a818a0a2cac61e067255611dbae8c410743e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-dodc721e/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
To: /content/quora_question_pairs_rus.csv.zip
100% 26.7M/26.7M [00:00<00:00, 90.3MB/s]
Archive:  quora_question_pairs

In [4]:
import pandas as pd

quora_df = pd.read_csv('quora_question_pairs_rus.csv', usecols=lambda x: 'Unnamed' not in x)
dataset_temp.extend(quora_df["question1"].values)
dataset_temp.extend(quora_df["question2"].values)
dataset = dataset_temp.copy()


In [5]:
import requests
from typing import List, Tuple


def check_spelling(text: str) -> str:
  domain = "https://speller.yandex.net/services/spellservice.json"
  words = text.split()
  if len(words) == 1:
    request = requests.get(domain + "/checkText?text=" + words[0])
    if requests:
      return request.json()[0]["word"], request.json()[0]["s"]
    else:
      return None
  elif len(words) > 1:
    words = "+".join(words)
    request = requests.get(domain + "/checkText?text=" + words)
    if requests:
      response = [(i["word"], i["s"]) for i in request.json()]
      if response:
        words_text = text.split()
        for elem in response:
          word, correct_words = elem
          try:
            words_text[words_text.index(word)] = correct_words[0]
          except:
            continue
        return " ".join(words_text).lower()
      else:
        return text
    else:
      return None
  return None
  

In [6]:
check_spelling("Левенштейн алгaритм")

'левенштейн алгоритм'

In [7]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 6.5 MB 30.2 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
model.cuda()

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()



Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
import numpy as np
from tqdm.notebook import tqdm


def cosine(u, v):
  res = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
  return res


def embed(text, batched=True):
    return embed_bert_cls(text, model, tokenizer)

In [10]:
from tqdm.notebook import tqdm

embedded_data = []
for x in tqdm(dataset):
  if type(x) == str:
    embedded_data += [{"sentence": x, "value": embed(x)}]



  0%|          | 0/810906 [00:00<?, ?it/s]

In [11]:
def get_result(text):
  query = embed(check_spelling(text), False)
  for idx, x in enumerate(embedded_data):
    embedded_data[idx]['cosine'] = cosine(x['value'], query)
  return sorted(embedded_data, key = lambda elem: elem['cosine'], reverse=True)[:10]

In [12]:
text = "Какой город самый большой в мире?"

for elem in get_result(text):
  print(elem['sentence'])

какой самый большой город в мире
какой самый большой город в мире
какой самый большой город в мире
какой самый большой город в Финляндии?
какой величайший город в мире
какой самый большой город в мире по площади
какой лучший город в мире
какой лучший город в мире
где самый красивый город в мире
где самый красивый город в мире


In [13]:
text = "Кокой город самй бальшой?"

for elem in get_result(text):
  print(elem['sentence'])

Какой океан самый маленький?
какой самый большой город
какой самый большой город
какой самый большой город
В какой стране город Бангкок?
Какой город является столицей Хорватии?
Какой лучший город для нас?
Какой город стоит на реке Печора?
Какой город является столицей Ботсваны?
В какой стране есть город Изюм?
